## Adjustment for Google Colab

In [ ]:
# mount drive for access to the
from google.colab import drive

In [ ]:
drive.mount("/content/drive")

In [ ]:
# all the drive the files are present in "/content/drive/My Drive"
!ls "/content/drive/My Drive/Beuth Uni/Master Thesis"

In [ ]:
import sys
sys.path.append('/content/drive/My Drive/Beuth Uni/Master Thesis/jenga')

In [ ]:
#! pip install openml

In [28]:
import numpy as np
from sklearn.linear_model import SGDClassifier

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

from jenga.basis import Dataset
from jenga.cleaning.ppp import PipelinePerformancePrediction
from jenga.cleaning.clean import Clean

In [2]:
seed = 10

## Dataset

In [3]:
dataset = Dataset(seed, "credit-g")

Dataset 'credit-g', target: 'class'
**Author**: Dr. Hans Hofmann  

**Source**: [UCI](https://archive.ics.uci.edu/ml/datasets/statlog+(german+credit+data)) - 1994    

**Please cite**: [UCI](https://archive.ics.uci.edu/ml/citation_policy.html)



**German Credit data**  

This dataset classifies people described by a set of attributes as good or bad credit risks.



This dataset comes with a cost matrix: 

``` 

      Good  Bad (predicted)  

Good   0    1   (actual)  

Bad    5    0  

```



It is worse to class a customer as go

Attribute types: 


,attribute_names,categorical_indicator
0,checking_status,True
1,duration,False
2,credit_history,True
3,purpose,True
4,credit_amount,False
5,savings_status,True
6,employment,True
7,installment_commitment,False
8,personal_status,True
9,other_parties,True


In [4]:
all_data = dataset.all_data
all_data

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,residence_since,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker,class
0,<0,6.0,critical/other existing credit,radio/tv,1169.0,no known savings,>=7,4.0,male single,none,4.0,real estate,67.0,none,own,2.0,skilled,1.0,yes,yes,good
1,0<=X<200,48.0,existing paid,radio/tv,5951.0,<100,1<=X<4,2.0,female div/dep/mar,none,2.0,real estate,22.0,none,own,1.0,skilled,1.0,none,yes,bad
2,no checking,12.0,critical/other existing credit,education,2096.0,<100,4<=X<7,2.0,male single,none,3.0,real estate,49.0,none,own,1.0,unskilled resident,2.0,none,yes,good
3,<0,42.0,existing paid,furniture/equipment,7882.0,<100,4<=X<7,2.0,male single,guarantor,4.0,life insurance,45.0,none,for free,1.0,skilled,2.0,none,yes,good
4,<0,24.0,delayed previously,new car,4870.0,<100,1<=X<4,3.0,male single,none,4.0,no known property,53.0,none,for free,2.0,skilled,2.0,none,yes,bad
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,no checking,12.0,existing paid,furniture/equipment,1736.0,<100,4<=X<7,3.0,female div/dep/mar,none,4.0,real estate,31.0,none,own,1.0,unskilled resident,1.0,none,yes,good
996,<0,30.0,existing paid,used car,3857.0,<100,1<=X<4,4.0,male div/sep,none,4.0,life insurance,40.0,none,own,1.0,high qualif/self emp/mgmt,1.0,yes,yes,good
997,no checking,12.0,existing paid,radio/tv,804.0,<100,>=7,4.0,male single,none,4.0,car,38.0,none,own,1.0,skilled,1.0,none,yes,good
998,<0,45.0,existing paid,radio/tv,1845.0,<100,1<=X<4,4.0,male single,none,4.0,no known property,23.0,none,for free,1.0,skilled,1.0,yes,yes,bad


In [5]:
attribute_names = dataset.attribute_names
attribute_names

['checking_status',
 'duration',
 'credit_history',
 'purpose',
 'credit_amount',
 'savings_status',
 'employment',
 'installment_commitment',
 'personal_status',
 'other_parties',
 'residence_since',
 'property_magnitude',
 'age',
 'other_payment_plans',
 'housing',
 'existing_credits',
 'job',
 'num_dependents',
 'own_telephone',
 'foreign_worker']

In [6]:
attribute_types = dataset.attribute_types
attribute_types

,attribute_names,categorical_indicator
0,checking_status,True
1,duration,False
2,credit_history,True
3,purpose,True
4,credit_amount,False
5,savings_status,True
6,employment,True
7,installment_commitment,False
8,personal_status,True
9,other_parties,True


### Categorical and Numerical Features

In [7]:
categorical_columns = dataset.categorical_columns
categorical_columns

['checking_status',
 'credit_history',
 'purpose',
 'savings_status',
 'employment',
 'personal_status',
 'other_parties',
 'property_magnitude',
 'other_payment_plans',
 'housing',
 'job',
 'own_telephone',
 'foreign_worker']

In [8]:
numerical_columns = dataset.numerical_columns
numerical_columns

['duration',
 'credit_amount',
 'installment_commitment',
 'residence_since',
 'age',
 'existing_credits',
 'num_dependents']

In [9]:
print(f"Found {len(categorical_columns)} categorical and {len(numerical_columns)} numeric features")

Found 13 categorical and 7 numeric features


## Model

### Model parameters

In [10]:
## model parameters
learner = SGDClassifier(max_iter=1000)
param_grid = {
    'learner__loss': ['log'],
    'learner__penalty': ['l2', 'l1', 'elasticnet'],
    'learner__alpha': [0.0001, 0.001, 0.01, 0.1]
}

### Preprocessing Pipeline

In [11]:
## preprocessing pipeline for both numerical and categorical columns

# preprocessing pipeline for numerical columns
transformer_numeric = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value=0)),
    ('standard_scale', StandardScaler())
])

# preprocessing pipeline for categorical columns
transformer_categorical = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='__NA__')),
    ('one_hot_encode', OneHotEncoder(handle_unknown='ignore'))
])

# preprocessor
feature_transform = ColumnTransformer(transformers=[
    ('categorical_features', transformer_categorical, categorical_columns),
    ('numerical_features', transformer_numeric, numerical_columns)
])

### Prediction Pipeline

In [12]:
## prediction pipeline: append classifier (learner) to the preprocessing pipeline
pipeline = Pipeline([
    ('features', feature_transform),
    ('learner', learner)
])

### Train and Test Data

In [13]:
train_data, train_labels, test_data, test_labels = dataset.get_train_test_data()

display(train_data.head())
print(train_labels[0:5])

display(test_data.head())
print(test_labels[0:5])

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,residence_since,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker
188,<0,12.0,existing paid,radio/tv,674.0,100<=X<500,4<=X<7,4.0,male mar/wid,none,1.0,life insurance,20.0,none,own,1.0,skilled,1.0,none,yes
194,0<=X<200,45.0,existing paid,radio/tv,3031.0,100<=X<500,1<=X<4,4.0,male single,guarantor,4.0,life insurance,21.0,none,rent,1.0,skilled,1.0,none,yes
225,no checking,36.0,no credits/all paid,repairs,2613.0,<100,1<=X<4,4.0,male single,none,2.0,car,27.0,none,own,2.0,skilled,1.0,none,yes
580,0<=X<200,18.0,critical/other existing credit,new car,1056.0,<100,>=7,3.0,male single,guarantor,3.0,real estate,30.0,bank,own,2.0,skilled,1.0,none,yes
428,no checking,9.0,existing paid,furniture/equipment,1313.0,<100,>=7,1.0,male single,none,4.0,car,20.0,none,own,1.0,skilled,1.0,none,yes


['bad' 'bad' 'good' 'bad' 'good']


,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,residence_since,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker
841,no checking,21.0,delayed previously,used car,2993.0,<100,1<=X<4,3.0,male single,none,2.0,real estate,28.0,stores,own,2.0,unskilled resident,1.0,none,yes
956,>=200,30.0,critical/other existing credit,radio/tv,3656.0,no known savings,>=7,4.0,male single,none,4.0,life insurance,49.0,stores,own,2.0,unskilled resident,1.0,none,yes
544,no checking,12.0,critical/other existing credit,new car,1255.0,<100,>=7,4.0,male single,none,4.0,real estate,61.0,none,own,2.0,unskilled resident,1.0,none,yes
173,0<=X<200,8.0,existing paid,radio/tv,1414.0,<100,1<=X<4,4.0,male single,guarantor,2.0,real estate,33.0,none,own,1.0,skilled,1.0,none,no
759,<0,12.0,critical/other existing credit,new car,691.0,<100,>=7,4.0,male single,none,3.0,life insurance,35.0,none,own,2.0,skilled,1.0,none,yes


['good' 'good' 'good' 'good' 'bad']


## PPP: Pipeline Performance Prediction

In [14]:
ppp = PipelinePerformancePrediction(seed, train_data, train_labels, test_data, test_labels, categorical_columns, numerical_columns, learner, param_grid, pipeline)

In [15]:
# model trained on original train data
ppp_model = ppp.fit_ppp(train_data)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    7.2s
[Parallel(n_jobs=-1)]: Done  53 out of  60 | elapsed:    7.4s remaining:    0.9s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    7.5s finished


In [16]:
# score on original test data
ppp_model_score = ppp.predict_score_ppp(ppp_model, test_data)
ppp_model_score

0.8093735390369332

## Corruptions

In [17]:
# generate corrpted test data
test_data_corrupted, perturbations, cols_perturbed = ppp.get_corrupted(test_data)

Generating corrupted training data on 200 rows...
Applying perturbations...
GaussianNoise: {'column': 'residence_since', 'fraction': 0.25} on column ['residence_since']
MissingValues: {'column': 'credit_history', 'fraction': 0.5, 'na_value': nan, 'missingness': 'MAR'} on column ['credit_history']
GaussianNoise: {'column': 'duration', 'fraction': 0.25} on column ['duration']
MissingValues: {'column': 'credit_amount', 'fraction': 0.75, 'na_value': nan, 'missingness': 'MAR'} on column ['credit_amount']
SwappedValues: {'column_a': 'housing', 'column_b': 'personal_status', 'fraction': 0.25} on column ['housing', 'personal_status']


## Cleaning

In [18]:
clean = Clean(train_data, test_data_corrupted, categorical_columns, numerical_columns, ppp, ppp_model)

In [19]:
test_data_cleaned, score_no_cleaning, cleaner_scores_ppp = clean(train_data, test_data_corrupted)

2020-06-23 23:46:39,593 [INFO]  CategoricalEncoder for column checking_status                                found only 44 occurrences of value >=200


PPP score no cleaning: 0.7850631136044881
PPP score with cleaning: <jenga.cleaning.cleaner.Cleaner object at 0x000002454A68EEF0>: 0.7850631136044881
PPP score with cleaning: <jenga.cleaning.cleaner.Cleaner object at 0x000002454A6A7C50>: 0.7911407199625993
Fitting model for column: checking_status


2020-06-23 23:46:40,439 [INFO]  
========== start: fit model
2020-06-23 23:46:40,440 [WARNING]  Already bound, ignoring bind()
2020-06-23 23:46:40,812 [INFO]  Epoch[0] Batch [0-23]	Speed: 1095.01 samples/sec	cross-entropy=1.273606	checking_status-accuracy=0.364583
2020-06-23 23:46:41,197 [INFO]  Epoch[0] Train-cross-entropy=1.233387
2020-06-23 23:46:41,199 [INFO]  Epoch[0] Train-checking_status-accuracy=0.368056
2020-06-23 23:46:41,200 [INFO]  Epoch[0] Time cost=0.753
2020-06-23 23:46:41,210 [INFO]  Saved checkpoint to "imputer_model\model-0000.params"
2020-06-23 23:46:41,271 [INFO]  Epoch[0] Validation-cross-entropy=1.030261
2020-06-23 23:46:41,272 [INFO]  Epoch[0] Validation-checking_status-accuracy=0.550000
2020-06-23 23:46:41,632 [INFO]  Epoch[1] Batch [0-23]	Speed: 1080.29 samples/sec	cross-entropy=1.132693	checking_status-accuracy=0.442708
2020-06-23 23:46:41,933 [INFO]  Epoch[1] Train-cross-entropy=1.134775
2020-06-23 23:46:41,934 [INFO]  Epoch[1] Train-checking_status-accuracy=

Fitting model for column: credit_history


2020-06-23 23:46:51,172 [INFO]  
========== start: fit model
2020-06-23 23:46:51,175 [WARNING]  Already bound, ignoring bind()
2020-06-23 23:46:51,602 [INFO]  Epoch[0] Batch [0-23]	Speed: 978.72 samples/sec	cross-entropy=1.097994	credit_history-accuracy=0.572917
2020-06-23 23:46:51,903 [INFO]  Epoch[0] Train-cross-entropy=1.055214
2020-06-23 23:46:51,905 [INFO]  Epoch[0] Train-credit_history-accuracy=0.619444
2020-06-23 23:46:51,906 [INFO]  Epoch[0] Time cost=0.727
2020-06-23 23:46:51,916 [INFO]  Saved checkpoint to "imputer_model\model-0000.params"
2020-06-23 23:46:51,976 [INFO]  Epoch[0] Validation-cross-entropy=0.863597
2020-06-23 23:46:51,977 [INFO]  Epoch[0] Validation-credit_history-accuracy=0.700000
2020-06-23 23:46:52,376 [INFO]  Epoch[1] Batch [0-23]	Speed: 973.51 samples/sec	cross-entropy=0.869200	credit_history-accuracy=0.705729
2020-06-23 23:46:52,729 [INFO]  Epoch[1] Train-cross-entropy=0.898504
2020-06-23 23:46:52,731 [INFO]  Epoch[1] Train-credit_history-accuracy=0.70138

...

Fitting model for column: existing_credits


2020-06-23 23:57:04,883 [INFO]  
========== start: fit model
2020-06-23 23:57:04,887 [WARNING]  Already bound, ignoring bind()
2020-06-23 23:57:05,218 [INFO]  Epoch[0] Batch [0-23]	Speed: 1182.63 samples/sec	cross-entropy=15.320334	existing_credits-accuracy=0.000000
2020-06-23 23:57:05,493 [INFO]  Epoch[0] Train-cross-entropy=15.455334
2020-06-23 23:57:05,499 [INFO]  Epoch[0] Train-existing_credits-accuracy=0.000000
2020-06-23 23:57:05,504 [INFO]  Epoch[0] Time cost=0.610
2020-06-23 23:57:05,514 [INFO]  Saved checkpoint to "imputer_model\model-0000.params"
2020-06-23 23:57:05,568 [INFO]  Epoch[0] Validation-cross-entropy=12.308671
2020-06-23 23:57:05,573 [INFO]  Epoch[0] Validation-existing_credits-accuracy=0.000000
2020-06-23 23:57:05,856 [INFO]  Epoch[1] Batch [0-23]	Speed: 1376.44 samples/sec	cross-entropy=13.204112	existing_credits-accuracy=0.000000
2020-06-23 23:57:06,111 [INFO]  Epoch[1] Train-cross-entropy=13.589545
2020-06-23 23:57:06,116 [INFO]  Epoch[1] Train-existing_credits

Fitting model for column: num_dependents


2020-06-23 23:57:13,275 [INFO]  
========== start: fit model
2020-06-23 23:57:13,284 [WARNING]  Already bound, ignoring bind()
2020-06-23 23:57:13,591 [INFO]  Epoch[0] Batch [0-23]	Speed: 1290.15 samples/sec	cross-entropy=14.863863	num_dependents-accuracy=0.000000
2020-06-23 23:57:13,859 [INFO]  Epoch[0] Train-cross-entropy=15.618117
2020-06-23 23:57:13,864 [INFO]  Epoch[0] Train-num_dependents-accuracy=0.000000
2020-06-23 23:57:13,870 [INFO]  Epoch[0] Time cost=0.578
2020-06-23 23:57:13,881 [INFO]  Saved checkpoint to "imputer_model\model-0000.params"
2020-06-23 23:57:13,936 [INFO]  Epoch[0] Validation-cross-entropy=14.198002
2020-06-23 23:57:13,941 [INFO]  Epoch[0] Validation-num_dependents-accuracy=0.000000
2020-06-23 23:57:14,240 [INFO]  Epoch[1] Batch [0-23]	Speed: 1315.07 samples/sec	cross-entropy=13.507775	num_dependents-accuracy=0.000000
2020-06-23 23:57:14,561 [INFO]  Epoch[1] Train-cross-entropy=14.474562
2020-06-23 23:57:14,569 [INFO]  Epoch[1] Train-num_dependents-accuracy=

PPP score with cleaning: <jenga.cleaning.cleaner.Cleaner object at 0x000002454A970668>: 0.78903693314633
Best cleaning method: <jenga.cleaning.cleaner.Cleaner object at 0x000002454A6A7C50>: 0.7911407199625993


## Results

### Model Results

In [21]:
# model 
pipeline.fit(train_data, train_labels)

Pipeline(memory=None,
         steps=[('features',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('categorical_features',
                                                  Pipeline(memory=None,
                                                           steps=[('imputer',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value='__NA__',
                                                                                 missing_values=nan,
                                                                                 strategy='constant',
                                          

In [22]:
# original data test score
pipeline.score(test_data, test_labels)

0.7

In [23]:
# corrupted data test score
pipeline.score(test_data_corrupted, test_labels)

0.69

In [24]:
# cleaned data test score
pipeline.score(test_data_cleaned, test_labels)

0.665

### PPP Results

In [25]:
# ppp model score
ppp_model_score

0.8093735390369332

In [26]:
# ppp score corrupted
score_no_cleaning

0.7850631136044881

In [29]:
# ppp score cleaned
np.array(cleaner_scores_ppp).max()

0.7911407199625993

In [30]:
# ppp cleaner scores
cleaner_scores_ppp

[0.7850631136044881,
 0.7911407199625993,
 0.7898550724637682,
 0.7850631136044881,
 0.7911407199625993,
 0.7897381954184197,
 0.7850631136044881,
 0.7911407199625993,
 0.78903693314633]